In [5]:
import nltk

nltk.download("punkt_tab")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
import random
import json
import nltk
from nltk.tokenize import word_tokenize
import pickle

class NepaliNeuralChatbot:
    def __init__(self):
        # Intents and patterns in Romanized Nepali
        self.intents = {
            "intents": [
                {
                    "tag": "delivery_time",
                    "patterns": [
                        "delivery kati time lagcha",
                        "mero order kahile aaucha",
                        "shipping time kati ho",
                        "parcel kati din ma aaucha"
                    ],
                    "responses": [
                        "Valley bhitra 2-3 din ma delivery huncha",
                        "Baira ko location ma 4-5 din lagcha",
                        "Express delivery ko lagi 24 ghanta lagcha"
                    ]
                },
                {
                    "tag": "payment_methods",
                    "patterns": [
                        "payment kasto kasto milcha",
                        "online payment cha",
                        "cash on delivery cha",
                        "esewa chalcha",
                        "khalti chalcha"
                    ],
                    "responses": [
                        "Hamro ma esewa, khalti, card payment ra COD cha",
                        "Online payment ra cash on delivery dubai cha",
                        "Sabai digital wallet haru support garcha"
                    ]
                },
                {
                    "tag": "refund",
                    "patterns": [
                        "refund policy k cha",
                        "return garna milcha",
                        "paisa firkine guarantee k cha",
                        "product man parena bhane k garne"
                    ],
                    "responses": [
                        "7 din bhitra return garna milcha",
                        "Original condition ma return garnu parcha",
                        "Return gareko 3 din bhitra refund huncha"
                    ]
                }
            ]
        }

        self.words = []
        self.classes = []
        self.documents = []
        self.ignore_words = ['?', '!', '.', ',']

    def preprocess_training_data(self):
        """Preprocess the training data"""
        # Extract all words from patterns
        for intent in self.intents['intents']:
            for pattern in intent['patterns']:
                # Tokenize each word
                w = word_tokenize(pattern)
                # Add to words list
                self.words.extend(w)
                # Add to documents
                self.documents.append((w, intent['tag']))
                # Add to classes list
                if intent['tag'] not in self.classes:
                    self.classes.append(intent['tag'])

        # Lower case and remove duplicates
        self.words = [w.lower() for w in self.words if w not in self.ignore_words]
        self.words = sorted(list(set(self.words)))
        self.classes = sorted(list(set(self.classes)))

    def create_training_data(self):
        """Create training data for neural network"""
        training = []
        output_empty = [0] * len(self.classes)

        # Create bag of words for each pattern
        for doc in self.documents:
            bag = []
            pattern_words = doc[0]
            pattern_words = [word.lower() for word in pattern_words]

            # Create bag of words array
            for w in self.words:
                bag.append(1) if w in pattern_words else bag.append(0)

            # Output is '1' for current tag and '0' for rest
            output_row = list(output_empty)
            output_row[self.classes.index(doc[1])] = 1
            training.append([bag, output_row])

        # Shuffle training data
        random.shuffle(training)
        training = np.array(training, dtype=object)

        # Split into X and y values
        train_x = list(training[:, 0])
        train_y = list(training[:, 1])

        return np.array(train_x), np.array(train_y)

    def build_model(self):
        """Build and train the neural network model"""
        # Create model - 3 layers
        model = Sequential()
        model.add(Dense(128, input_shape=(len(self.words),), activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(len(self.classes), activation='softmax'))

        # Compile model
        model.compile(loss='categorical_crossentropy',
                     optimizer='adam',
                     metrics=['accuracy'])

        return model

    def train(self):
        """Train the chatbot"""
        self.preprocess_training_data()
        X, y = self.create_training_data()

        # Build and train the model
        self.model = self.build_model()
        self.model.fit(X, y, epochs=200, batch_size=5, verbose=1)

    def clean_up_sentence(self, sentence):
        """Tokenize and clean up input sentence"""
        sentence_words = word_tokenize(sentence)
        sentence_words = [word.lower() for word in sentence_words]
        return sentence_words

    def bow(self, sentence):
        """Convert sentence to bag of words"""
        sentence_words = self.clean_up_sentence(sentence)
        bag = [0] * len(self.words)
        for s in sentence_words:
            for i, w in enumerate(self.words):
                if w == s:
                    bag[i] = 1
        return np.array(bag)

    def predict_class(self, sentence):
        """Predict the class of the input sentence"""
        p = self.bow(sentence)
        res = self.model.predict(np.array([p]))[0]
        ERROR_THRESHOLD = 0.25
        results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
        results.sort(key=lambda x: x[1], reverse=True)
        return_list = []
        for r in results:
            return_list.append({"intent": self.classes[r[0]], "probability": str(r[1])})
        return return_list

    def get_response(self, ints):
        """Get response based on predicted intent"""
        if not ints:
            return "Maaf garnuhos, ma tapai ko prashna bujhina."

        tag = ints[0]['intent']
        intents_list = self.intents['intents']
        for i in intents_list:
            if i['tag'] == tag:
                return random.choice(i['responses'])

        return "Maaf garnuhos, ma tapai ko prashna bujhina."

    def chat(self, message):
        """Main chat function"""
        ints = self.predict_class(message)
        return self.get_response(ints)

    def save_model(self, model_path='model.h5', data_path='training_data.pkl'):
        """Save the trained model and training data"""
        self.model.save(model_path)
        training_data = {
            'words': self.words,
            'classes': self.classes,
            'intents': self.intents
        }
        with open(data_path, 'wb') as f:
            pickle.dump(training_data, f)

    def load_model(self, model_path='model.h5', data_path='training_data.pkl'):
        """Load a trained model and training data"""
        self.model = tf.keras.models.load_model(model_path)
        with open(data_path, 'rb') as f:
            training_data = pickle.load(f)
        self.words = training_data['words']
        self.classes = training_data['classes']
        self.intents = training_data['intents']

# Example usage
chatbot = NepaliNeuralChatbot()
chatbot.train()

# Test the chatbot
test_queries = [
    "delivery kati time lagcha?",
    "payment kasto kasto milcha?",
    "return policy k cha?",
    "esewa chalcha?",
    "delivery ko subidha kasto xa",
    "kasari payment garna milcha"
]

for query in test_queries:
    response = chatbot.chat(query)
    print(f"Query: {query}")
    print(f"Response: {response}\n")

Epoch 1/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.4942 - loss: 1.1156
Epoch 2/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2404 - loss: 1.1859 
Epoch 3/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4692 - loss: 1.0426 
Epoch 4/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3558 - loss: 1.0780 
Epoch 5/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3423 - loss: 1.0683 
Epoch 6/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5962 - loss: 1.0283 
Epoch 7/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6077 - loss: 0.9711 
Epoch 8/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4673 - loss: 1.0178 
Epoch 9/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7346 - loss: 0.9738 
Epoch 10/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5827 - loss: 0.9471 
Epoch 11/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5192 - loss: 0.9353 
Epoch 12/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7596 - loss

In [11]:
# Test the chatbot
test_queries = [
    "delivery kati time lagcha?",
    "payment kasto kasto milcha?",
    "return policy k cha?",
    "esewa chalcha?",
    "delivery ko subidha kasto xa",
    "kasari payment garna milcha",
    "payment kasari garne"
]

for query in test_queries:
    response = chatbot.chat(query)
    print(f"Query: {query}")
    print(f"Response: {response}\n")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Query: delivery kati time lagcha?
Response: Express delivery ko lagi 24 ghanta lagcha

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Query: payment kasto kasto milcha?
Response: Online payment ra cash on delivery dubai cha

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Query: return policy k cha?
Response: Return gareko 3 din bhitra refund huncha

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Query: esewa chalcha?
Response: Online payment ra cash on delivery dubai cha

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Query: delivery ko subidha kasto xa
Response: Online payment ra cash on delivery dubai cha

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Query: kasari payment garna milcha
Response: Hamro ma esewa, khalti, card payment ra COD cha

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Query: payment kasari garne
Response: Sabai digital wallet haru support garcha

